In [1]:
import pandas as pd
import numpy as np

import sys
display(sys.version)

'3.11.5 (main, Sep 11 2023, 13:54:46) [GCC 11.2.0]'

In [2]:
import pomegranate
from pomegranate import *
import anndata

display(pomegranate.__version__)

'0.14.8'

# Import hDel-v3 (low MOI) & hDel-v4 (high MOI) bustools-filtered sgRNA UMIs

In [33]:
path = '/home/tyler/Documents/kallisto_bustools_kITE/'

kb3 = anndata.read_h5ad(path + 'hDel-v3_sgRNA_kITE_counts_filtered_concatenate.h5ad')
kb4 = anndata.read_h5ad(path + 'hDel-v4_sgRNA_kITE_counts_filtered_concatenate.h5ad')

kb3, kb4

(AnnData object with n_obs × n_vars = 25589 × 160
     obs: 'batch'
     var: 'feature_name',
 AnnData object with n_obs × n_vars = 19126 × 923
     obs: 'batch'
     var: 'feature_name')

In [34]:
kb_df3, kb_df4 = kb3.to_df(), kb4.to_df()

In [35]:
#sgRNAs must have at least 5 UMIs in at least 50 Cell Barcodes

kb_df3 = kb_df3.loc[:, (kb_df3 >= 5).sum() >= 50]
kb_df4 = kb_df4.loc[:, (kb_df4 >= 5).sum() >= 50]

# Poisson-Gaussian mixture model from Replogle et al., 2020

In [36]:
def PoissonGaussian(sgRNA: str, df, UMI_threshold):
    
    print(sgRNA)
    cell_barcodes = df[sgRNA].where(lambda x: x > UMI_threshold).dropna().index
    UMIs = df[sgRNA].where(lambda x: x > UMI_threshold).dropna().values
    log2_UMIs = np.log2(df[sgRNA].where(lambda x: x > UMI_threshold).dropna()).values.reshape(-1,1)
    
    i = 0
    #array of 1000 linearly spaced points between -2 and max(log2_UMIs)+2
    gmm_x = np.linspace(-2, max(log2_UMIs)+2, 1000)
    #run until variable i != 0
    while i == 0:
        #create a model using the GeneralMixtureModel.from_samples function, which is a mixture model of two distributions: PoissonDistribution and NormalDistribution
        model = GeneralMixtureModel.from_samples([PoissonDistribution, NormalDistribution], 2, log2_UMIs)
        #check that the model converges (no NaN) and that the Poisson distribution is the lower component by using an if-else loop
        #if there is any NaN in the probability of the model, set i to 0 to keep the loop running
        if numpy.isnan(model.probability(gmm_x)).any():
            i = 0
        else:
            #check that Poisson distribution is the lower component
            if model.distributions[0].name is 'PoissonDistribution':
                #if the first component of the model is PoissonDistribution and has a smaller parameter than the second component, set i to 1 to exit the loop
                if model.distributions[0].parameters[0] < model.distributions[1].parameters[0]:
                    i = 1    
                else:
                    i = 0
            #if the first component has a bigger parameter than the second, set i to 0 to keep the loop running
            elif model.distributions[0].parameters[0] > model.distributions[1].parameters[0]:
                i = 0
    
    #second column is probability of belonging to second component
    return pd.DataFrame({'cell_barcode': cell_barcodes, 'sg_ID': sgRNA, 'UMIs': UMIs, 'PoissonGaussian': model.predict_proba(log2_UMIs)[:,1]})

<>:21: SyntaxWarning: "is" with a literal. Did you mean "=="?
<>:21: SyntaxWarning: "is" with a literal. Did you mean "=="?
/tmp/ipykernel_422324/3385834778.py:21: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if model.distributions[0].name is 'PoissonDistribution':


# Perform mixture model sgRNA to cell assignments

In [39]:
#drop sgRNAs for which the mixture model fails to converge (very few UMIs, do belong to a mixture of Poisson-Gaussian distributions)

no_converge = ['hDel_1273_1_H3K4me1_H3K27ac_CUT_Tag_4', 'hDel_3813_1_H3K4me1_H3K27ac_CUT_Tag_5', 'hDel_4959_1_Omni_ATAC_3', 
               'hDel_5502_1_Omni_ATAC_5', 'hDel_6065_1_Omni_ATAC_3', 'hDel_6111_1_Omni_ATAC_3', 
               'hDel_4959_1_Omni_ATAC_4', 'hDel_6602_1_Omni_ATAC_3', 'hDel_5049_1_Omni_ATAC_2']

kb_df4.drop(columns=no_converge, errors='ignore', inplace=True)

In [41]:
#perform sgRNA-cell assignments

assignments_kb3 = [PoissonGaussian(sgRNA, kb_df3, 1) for sgRNA in kb_df3.columns]
assignments_kb4 = [PoissonGaussian(sgRNA, kb_df4, 1) for sgRNA in kb_df4.columns]

PGMM_sgRNAs_kb3 = pd.concat(assignments_kb3)
PGMM_sgRNAs_kb4 = pd.concat(assignments_kb4)

hDel_1611:41
hDel_1611:43
hDel_1611:46
hDel_1611:47
hDel_1611:49
hDel_3130:28
hDel_3130:37
hDel_3130:38
hDel_349:13
hDel_349:34
hDel_349:75
hDel_349:77
hDel_349:81
hDel_349:82
hDel_349:89
hDel_349:90
hDel_349:91
hDel_349:92
hDel_349:94
hDel_349:95
hDel_349:96
hDel_349:98
hDel_349:104
hDel_349:162
hDel_361:154
hDel_361:163
hDel_361:168
hDel_4859:198
hDel_4859:199
hDel_4859:203
hDel_4859:204
hDel_5068:25
hDel_5068:29
hDel_5068:30
hDel_5068:36
hDel_5068:37
hDel_5068:44
hDel_5286:113
hDel_5286:127
hDel_5286:128
hDel_5286:129
hDel_5286:142
hDel_5488:94
hDel_5488:102
hDel_5488:107
hDel_5488:110
hDel_5488:113
hDel_5669:100
hDel_5669:109
hDel_5669:111
hDel_5669:116
hDel_5689:61
hDel_5689:62
hDel_5689:78
hDel_5689:124
hDel_5689:147
hDel_5689:150
hDel_5689:196
hDel_5689:205
hDel_5689:228
hDel_5689:259
hDel_5689:260
hDel_5689:279
hDel_5689:290
hDel_5689:331
hDel_5689:341
hDel_5689:345
hDel_5689:357
hDel_5689:358
hDel_5689:365
hDel_5689:372
hDel_5736:72
hDel_5736:73
hDel_5736:77
hDel_5738:147
hDel

# Filter sgRNA-cell assignments

In [10]:
#filters: 
    #>99% probability of belonging to second component
    #>10 UMIs
    #1 sgRNA per cell
    #>25 cells per sgRNA

assigned_sgRNAs_filtered = assigned_sgRNAs.loc[(assigned_sgRNAs['PoissonGaussian'] >= 0.99) & (assigned_sgRNAs['UMIs'] >= 10)].groupby('cell_barcode').filter(lambda x: len(x) == 1).groupby('sg_ID').filter(lambda x: len(x) >= 25)
assigned_sgRNAs_filtered

,cell_barcode,sg_ID,UMIs,PoissonGaussian
1,AAAGAACTCGAGTGGA_GEM1,hDel_1611:41,3417.0,1.000000
3,AACAAAGGTCCAGCAC_GEM1,hDel_1611:41,3624.0,1.000000
6,AAGACTCCAGGCACTC_GEM1,hDel_1611:41,866.0,0.999998
7,AATAGAGGTTCGAGCC_GEM1,hDel_1611:41,4370.0,1.000000
9,ACCCAAAGTAAGATCA_GEM1,hDel_1611:41,2454.0,1.000000
...,...,...,...,...
967,TCCTAATGTTGCAAGG_GEM5,Gasperini_chr1.4082_2,1096.0,0.999811
968,TCCTCCCCAATTGTGC_GEM5,Gasperini_chr1.4082_2,810.0,0.999660
985,TGTTCATCAGATACTC_GEM5,Gasperini_chr1.4082_2,1701.0,0.999900
987,TTAGGGTTCTCTTGCG_GEM5,Gasperini_chr1.4082_2,1133.0,0.999821


In [24]:
#filters: 
    #>50% probability of belonging to second component
    #>10 UMIs
    #>100 cells per sgRNA

assigned_sgRNAs_filtered = assigned_sgRNAs.loc[(assigned_sgRNAs['PoissonGaussian'] > 0.50) & (assigned_sgRNAs['UMIs'] >= 10)].groupby('sg_ID').filter(lambda x: len(x) >= 100)
assigned_sgRNAs_filtered

,cell_barcode,sg_ID,UMIs,PoissonGaussian
0,ACAAGCTTCGAGCACC_GEM1,hDel_1018_1_H3K4me1_H3K27ac_CUT_Tag_3,1718.0,0.999991
1,ACTATGGGTGAAGCGT_GEM1,hDel_1018_1_H3K4me1_H3K27ac_CUT_Tag_3,1922.0,0.999992
3,ATTACCTGTAGTCGGA_GEM1,hDel_1018_1_H3K4me1_H3K27ac_CUT_Tag_3,1249.0,0.999986
4,ATTGTTCAGACCAAAT_GEM1,hDel_1018_1_H3K4me1_H3K27ac_CUT_Tag_3,725.0,0.999960
5,ATTGTTCCATGACTTG_GEM1,hDel_1018_1_H3K4me1_H3K27ac_CUT_Tag_3,855.0,0.999972
...,...,...,...,...
410,TTCTAGTAGTAGAGTT_GEM5,ReplogleCTRL_03043,219.0,0.996499
411,TTCTCTCCATGAGAAT_GEM5,ReplogleCTRL_03043,91.0,0.775577
412,TTGACCCAGTGGTGGT_GEM5,ReplogleCTRL_03043,455.0,0.999801
413,TTGAGTGTCGACGTCG_GEM5,ReplogleCTRL_03043,786.0,0.999962
